# Part 1

--- Day 11: Seating System ---

Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (`.`), an empty seat (`L`), or an occupied seat (`#`). For example, the initial seat layout might look like this:

```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
```

Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the number of occupied seats adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:

- If a seat is empty (`L`) and there are no occupied seats adjacent to it, the seat becomes occupied.
- If a seat is occupied (`#`) and four or more seats adjacent to it are also occupied, the seat becomes empty.
- Otherwise, the seat's state does not change.
- Floor (`.`) never changes; seats don't move, and nobody sits on the floor.

After one round of these rules, every seat in the example layout becomes occupied:

```
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
```

After a second round, the seats with four or more occupied adjacent seats become empty again:

```
#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##
```

This process continues for three more rounds:

```
#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##
```

```
#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##
```

```
#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##
```

At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count 37 occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?

In [2]:
from pathlib import Path

import numba
import numpy as np

In [3]:
INPUT_FILE = Path.cwd() / 'inputs' / 'day11' / 'part1.txt'

In [4]:
CHAR_MAP = {'.': 0, 'L': 1, '#': 2}

In [5]:
def input_lines():
    with INPUT_FILE.open() as fp:
        for line in fp:
            yield line.strip()

In [6]:
def input_array():
    return np.array(
        tuple(
            (tuple(CHAR_MAP[char] for char in line)
            for line in input_lines())
        ),
        dtype=np.uint8
    )

In [7]:
input_array().shape

(97, 94)

In [8]:
@numba.jit(nopython=True, nogil=True)
def adjacent_occupied(array, row, col):
    row_start = max(0, row - 1)
    row_end = min(row + 2, array.shape[0])
    col_start = max(0, col - 1)
    col_end = min(col + 2, array.shape[1])
    
    count = 0
    
    for r in range(row_start, row_end):
        for c in range(col_start, col_end):
            if array[r, c] == 2 and not (r == row and c == col):
                count += 1
    
    return count

In [9]:
adjacent_occupied(np.uint8([[2, 2, 2], [2, 2, 2], [2, 2, 2]]), 1, 1)

8

In [10]:
@numba.jit(nopython=True, nogil=True)
def new_value(array, row, col):
    current_val = array[row, col]
    if current_val == 0:
        return 0
    
    occupied = adjacent_occupied(array, row, col)
    if current_val == 1:
        if occupied == 0:
            return 2
        else:
            return 1
    else:
        if occupied >= 4:
            return 1
        else:
            return 2

In [11]:
@numba.jit(nopython=True, nogil=True)
def do_cycle(array, new_array):
    for row in range(array.shape[0]):
        for col in range(array.shape[1]):
            new_array[row, col] = new_value(array, row, col)

In [12]:
@numba.jit(nopython=True, nogil=True)
def run_model(array1):
    array2 = np.zeros_like(array1)
    while True:
        do_cycle(array1, array2)
        if np.array_equal(array1, array2):
            return array1
        array1, array2 = array2, array1

In [13]:
result = run_model(input_array())

In [14]:
(result == 2).sum()

2368

# Part 2

As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about the first seat they can see in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider the first seat in each of those eight directions. For example, the empty seat below would see eight occupied seats:

```
.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
```

The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:

```
.............
.L.L.#.#.#.#.
.............
```

The empty seat below would see no occupied seats:

```
.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
```

Also, people seem to be more tolerant than you expected: it now takes five or more visible occupied seats for an occupied seat to become empty (rather than four or more from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.

Given the same starting layout as above, these new rules cause the seating area to shift around as follows:

```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
```

```
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
```

```
#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#
```

```
#.L#.##.L#
#L#####.LL
L.#.#..#..
##L#.##.##
#.##.#L.##
#.#####.#L
..#.#.....
LLL####LL#
#.L#####.L
#.L####.L#
```

```
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##LL.LL.L#
L.LL.LL.L#
#.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLL#.L
#.L#LL#.L#
```

```
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.#L.L#
#.L####.LL
..#.#.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
```

```
#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
```

Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count 26 occupied seats.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?

In [29]:
@numba.jit(nopython=True, nogil=True)
def occupied_seat_north(array, row, col):
    # row -1, col 0
    for r in range(row - 1, -1, -1):
        if array[r, col] == 2:
            return True
        elif array[r, col] == 1:
            return False
    return False

In [30]:
@numba.jit(nopython=True, nogil=True)
def occupied_seat_south(array, row, col):
    # row +1, col 0
    for r in range(row + 1, array.shape[0]):
        if array[r, col] == 2:
            return True
        elif array[r, col] == 1:
            return False
    return False

In [31]:
@numba.jit(nopython=True, nogil=True)
def occupied_seat_west(array, row, col):
    # row 0, col -1
    for c in range(col - 1, -1, -1):
        if array[row, c] == 2:
            return True
        elif array[row, c] == 1:
            return False
    return False

In [32]:
@numba.jit(nopython=True, nogil=True)
def occupied_seat_east(array, row, col):
    # row 0, col +1
    for c in range(col + 1, array.shape[1]):
        if array[row, c] == 2:
            return True
        elif array[row, c] == 1:
            return False
    return False

In [33]:
@numba.jit(nopython=True, nogil=True)
def occupied_seat_ne(array, row, col):
    # row -1, col +1
    for r, c in zip(range(row - 1, -1, -1), range(col + 1, array.shape[1])):
        if array[r, c] == 2:
            return True
        elif array[r, c] == 1:
            return False
    return False

In [34]:
@numba.jit(nopython=True, nogil=True)
def occupied_seat_se(array, row, col):
    # row +1, col +1
    for r, c in zip(range(row + 1, array.shape[0]), range(col + 1, array.shape[1])):
        if array[r, c] == 2:
            return True
        elif array[r, c] == 1:
            return False
    return False

In [35]:
@numba.jit(nopython=True, nogil=True)
def occupied_seat_sw(array, row, col):
    # row +1, col -1
    for r, c in zip(range(row + 1, array.shape[0]), range(col - 1, -1, -1)):
        if array[r, c] == 2:
            return True
        elif array[r, c] == 1:
            return False
    return False

In [36]:
@numba.jit(nopython=True, nogil=True)
def occupied_seat_nw(array, row, col):
    # row -1, col -1
    for r, c in zip(range(row - 1, -1, -1), range(col - 1, -1, -1)):
        if array[r, c] == 2:
            return True
        elif array[r, c] == 1:
            return False
    return False

In [37]:
@numba.jit(nopython=True, nogil=True)
def visible_occupied_seats(array, row, col):
    count = 0
    
    if occupied_seat_north(array, row, col):
        count += 1
    if occupied_seat_south(array, row, col):
        count += 1
    if occupied_seat_east(array, row, col):
        count += 1
    if occupied_seat_west(array, row, col):
        count += 1
    if occupied_seat_ne(array, row, col):
        count += 1
    if occupied_seat_se(array, row, col):
        count += 1
    if occupied_seat_sw(array, row, col):
        count += 1
    if occupied_seat_nw(array, row, col):
        count += 1
    
    return count

In [38]:
@numba.jit(nopython=True, nogil=True)
def new_value2(array, row, col):
    current_val = array[row, col]
    if current_val == 0:
        return 0
    
    occupied = visible_occupied_seats(array, row, col)
    if current_val == 1:
        if occupied == 0:
            return 2
        else:
            return 1
    else:
        if occupied >= 5:
            return 1
        else:
            return 2

In [39]:
@numba.jit(nopython=True, nogil=True)
def do_cycle2(array, new_array):
    for row in range(array.shape[0]):
        for col in range(array.shape[1]):
            new_array[row, col] = new_value2(array, row, col)

In [40]:
@numba.jit(nopython=True, nogil=True)
def run_model2(array1):
    array2 = np.zeros_like(array1)
    while True:
        do_cycle2(array1, array2)
        if np.array_equal(array1, array2):
            return array1
        array1, array2 = array2, array1

In [41]:
result2 = run_model2(input_array())

In [42]:
(result2 == 2).sum()

2124